# Imports

In [ ]:
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

# Model & Tokenizer

## Import model and tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

# Data

In [ ]:
input_texts = ['query: how much protein should a female eat',
               'query: I like the blue sky',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: The sky is blue",
               "passage: I like to eat pizza"]

# Test

In [ ]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
print(batch_dict)

In [24]:
outputs = model(**batch_dict)
print(outputs)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 1.1787, -0.2960, -1.1996,  ..., -0.5630, -1.3796,  1.0059],
         [ 0.5923, -0.4263, -0.8927,  ..., -0.4820, -1.1604,  0.7519],
         [ 0.5456, -0.6482, -1.2272,  ..., -0.7752, -1.3413,  0.6554],
         ...,
         [ 1.1787, -0.2960, -1.1996,  ..., -0.5630, -1.3796,  1.0059],
         [ 1.1787, -0.2960, -1.1996,  ..., -0.5630, -1.3796,  1.0059],
         [ 1.1787, -0.2960, -1.1996,  ..., -0.5630, -1.3796,  1.0059]],

        [[ 0.6856, -0.7174, -1.6397,  ..., -0.3560, -0.4480,  0.6896],
         [ 0.3218, -0.4208, -0.8291,  ..., -0.1230, -0.2352,  0.1965],
         [ 0.3348, -0.3418, -1.4797,  ..., -0.2605,  0.3100, -0.1354],
         ...,
         [ 0.6856, -0.7174, -1.6397,  ..., -0.3560, -0.4480,  0.6896],
         [ 0.6856, -0.7174, -1.6397,  ..., -0.3560, -0.4480,  0.6896],
         [ 0.6856, -0.7174, -1.6397,  ..., -0.3560, -0.4480,  0.6896]],

        [[ 1.4116, -0.5168, -1.6717,  ..., -0.5477, -

In [ ]:
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
print(embeddings)

In [ ]:
embeddings = F.normalize(embeddings, p=2, dim=1)
print(embeddings)

In [ ]:
# normalize embeddings
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())